In [1]:
sc
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [2]:
flightsPath = "hdfs://localhost:9000/spark/rdd/flights.csv"

In [3]:
flightsData = sc.textFile(flightsPath)

In [4]:
flightsData.first()

u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",'

In [5]:
#  notHeader()    function to remove Header


def notHeader(row):
    return "AIRLINE_ID" not in row

In [6]:
# split()

import csv 
from StringIO import StringIO

def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

In [7]:
d1 = flightsData.filter(notHeader)
d1.take(3)

[u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,']

In [8]:
d2 = d1.map(split)
d2.take(2)

[['2016-01-01',
  '19790',
  '1248',
  'DTW',
  'LAX',
  '1935',
  '0.00',
  '2120',
  '-24.00',
  '249.00',
  '1979.00',
  ''],
 ['2016-01-01',
  '19790',
  '1251',
  'ATL',
  'GRR',
  '2130',
  '5.00',
  '2319',
  '-2.00',
  '92.00',
  '640.00',
  '']]

In [9]:
from datetime import datetime 

d3 = d2.map(lambda x: Row(date= datetime.strptime(x[0], "%Y-%m-%d"), airline = x[1], flightnum=x[2], origin=x[3], dest=x[4], dep=x[5],
                                                              dep_delay=x[6], arv=x[7], arv_delay=x[8],
                                                               airtime=x[9], distance=float(x[10]) ))

#7.Write a pySpark Code to Calculate Total No. of Flights Departed From Airport “LAX” on ‘2016-01-01’


In [10]:

d3.\
    filter(lambda x:(x['date'] == datetime.strptime("2016-01-01", "%Y-%m-%d") )and (x['dest']=='LAX')).\
    count()

508

#8.Write a pySpark Code to Clean Data By Using User Defined Function, Handle the same namedTuple Function and Padded all blank Fields with “00.00”.

In [11]:
sc

In [12]:
flightsPath = "hdfs://localhost:9000/spark/rdd/flights.csv"

In [13]:
flightsData = sc.textFile(flightsPath)

In [14]:
Padded all blank Fields with “00.00”
blanks = flightsData.map(lambda x:','.join(x or '00.00' for x in x.split(',')))

In [16]:
blanks.take(10)

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",00.00',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,00.00',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,00.00',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,00.00',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,00.00',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,00.00',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,00.00',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,00.00',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,00.00',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,00.00']

In [17]:
#Clean Data By Using User Defined Function
def notHeader(row):
    return "AIRLINE_ID" not in row

In [18]:
d1 = flightsData.filter(notHeader)
d1.take(3)

[u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,']

In [19]:
#Handle the same namedTuple Function


from datetime import datetime 
from collections import namedtuple

fields   = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep',
            'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

Flight   = namedtuple('Flight', fields, verbose=False)

DATE_FMT = '%Y-%m-%d'

TIME_FMT = '%H%M%S' # %S is added to remove error "unconverted data remains 0 " in strptime()


def parse(row):
    row[0]  = datetime.strptime(row[0], DATE_FMT).date()
    row[5]  = datetime.strptime(row[5], TIME_FMT).time()
    row[6]  = float(row[6])
    row[7]  = datetime.strptime(row[7], TIME_FMT).time()
    row[8]  = float(row[8])
    row[9]  = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

In [23]:
import csv 
from StringIO import StringIO

def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

In [28]:
# replacing blank time data with 0000 in required format as "0000"

blanktime = blanks.map(lambda x:x.replace(',""' , ',"0000"') )

In [29]:
finalF = blanktime                

In [30]:
finalF.take(242)

[u'"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",00.00',
 u'2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,00.00',
 u'2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,00.00',
 u'2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,00.00',
 u'2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,00.00',
 u'2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,00.00',
 u'2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,00.00',
 u'2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,00.00',
 u'2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,00.00',
 u'2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,00.00',
 u'2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,00.00',
 u'20

In [31]:
f = finalF.filter(notHeader).map(split)

In [32]:
f.take(717)

[['2016-01-01',
  '19790',
  '1248',
  'DTW',
  'LAX',
  '1935',
  '0.00',
  '2120',
  '-24.00',
  '249.00',
  '1979.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1251',
  'ATL',
  'GRR',
  '2130',
  '5.00',
  '2319',
  '-2.00',
  '92.00',
  '640.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1254',
  'LAX',
  'ATL',
  '2256',
  '1.00',
  '0547',
  '-13.00',
  '207.00',
  '1947.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1255',
  'SLC',
  'ATL',
  '1700',
  '4.00',
  '2213',
  '-16.00',
  '173.00',
  '1590.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1256',
  'BZN',
  'MSP',
  '1012',
  '72.00',
  '1420',
  '124.00',
  '121.00',
  '874.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1257',
  'ATL',
  'BNA',
  '1356',
  '83.00',
  '1402',
  '83.00',
  '38.00',
  '214.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1257',
  'BNA',
  'ATL',
  '1446',
  '86.00',
  '1644',
  '74.00',
  '37.00',
  '214.00',
  '00.00'],
 ['2016-01-01',
  '19790',
  '1258',
  'ATL',
  'JAX',
  '0946',
  '1

In [33]:
fp = f.map(parse)

In [34]:
fp.take(5)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1251', origin='ATL', dest='GRR', dep=datetime.time(21, 3), dep_delay=5.0, arv=datetime.time(23, 1, 9), arv_delay=-2.0, airtime=92.0, distance=640.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1255', origin='SLC', dest='ATL', dep=datetime.time(17, 0), dep_delay=4.0, arv=datetime.time(22, 1, 3), arv_delay=-16.0, airtime=173.0, distance=1590.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1256', origin='BZN', dest='MSP', dep=datetime.tim

#6 Write a Spark RDD code to get average delay in flights.

In [35]:

sumCount=fp.map(lambda x:x.dep_delay).aggregate((0,0),
                                                          (lambda acc,value: (acc[0]+value, acc[1]+1)),
                                                          (lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])))

In [36]:
print "The average delay is "+str(sumCount[0]/float(sumCount[1]))

The average delay is 7.59297664789


4 Write a Spark sql code to convert json data and select query where screen_user.name-realDonaldTrump

In [37]:
twitterPath="hdfs://localhost:9000/spark/cache-0.json"

In [38]:
import json
twitterData=sc.textFile(twitterPath).map(lambda x:json.loads(x))

In [39]:
twitterData.take(1)

[{u'contributors': None,
  u'coordinates': None,
  u'created_at': u'Sun Sep 09 21:17:55 +0000 2012',
  u'entities': {u'hashtags': [{u'indices': [87, 95], u'text': u'florida'}],
   u'urls': [{u'display_url': u'STLtoday.com',
     u'expanded_url': u'http://STLtoday.com',
     u'indices': [45, 65],
     u'url': u'http://t.co/OcISvreb'},
    {u'display_url': u'bit.ly/Tzhmly',
     u'expanded_url': u'http://bit.ly/Tzhmly',
     u'indices': [66, 86],
     u'url': u'http://t.co/FsJ7xgGW'}],
   u'user_mentions': []},
  u'favorited': False,
  u'geo': None,
  u'id': 244907511377965056,
  u'id_str': u'244907511377965056',
  u'in_reply_to_screen_name': None,
  u'in_reply_to_status_id': None,
  u'in_reply_to_status_id_str': None,
  u'in_reply_to_user_id': None,
  u'in_reply_to_user_id_str': None,
  u'place': None,
  u'possibly_sensitive': False,
  u'possibly_sensitive_editable': True,
  u'retweet_count': 0,
  u'retweeted': False,
  u'source': u'<a href="http://twitterfeed.com" rel="nofollow">twitte

In [40]:
twitterData.\
    filter(lambda x:x['user']['screen_name']=="realDonaldTrump").\
    map(lambda x:x['text'])\
    .take(10) #this is difficult to write

[u'Obama asked a 7 yr old for his birth certificate. He\'s "in your face" because the Republicans dropped the ball. (cont) http://t.co/FufZD79U',
 u'Obama is taunting the Republicans on the birther issue. They should call his bluff &amp; demand the REAL facts. He (cont) http://t.co/NWmVp06e',
 u'"President Obama is the greatest hoax ever perpetrated on the American people"\n--Clint Eastwood']

In [41]:
from pyspark.sql import SQLContext, Row

In [42]:
sqlC=SQLContext(sc)

In [43]:
sqlC

In [44]:
twitterTable=sqlC.read.json(twitterPath)

In [45]:
twitterTable.registerTempTable("twitterTable")

In [46]:
sqlC.sql("Select text, user.screen_name from twitterTable where user.screen_name='realDonaldTrump' limit 10").collect()

[Row(text=u'Obama asked a 7 yr old for his birth certificate. He\'s "in your face" because the Republicans dropped the ball. (cont) http://t.co/FufZD79U', screen_name=u'realDonaldTrump'),
 Row(text=u'Obama is taunting the Republicans on the birther issue. They should call his bluff &amp; demand the REAL facts. He (cont) http://t.co/NWmVp06e', screen_name=u'realDonaldTrump'),
 Row(text=u'"President Obama is the greatest hoax ever perpetrated on the American people"\n--Clint Eastwood', screen_name=u'realDonaldTrump')]

#3.Write a Pyspark code to perform Action like (take,collect,count and first)

In [47]:
fp.take(10)

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1251', origin='ATL', dest='GRR', dep=datetime.time(21, 3), dep_delay=5.0, arv=datetime.time(23, 1, 9), arv_delay=-2.0, airtime=92.0, distance=640.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1255', origin='SLC', dest='ATL', dep=datetime.time(17, 0), dep_delay=4.0, arv=datetime.time(22, 1, 3), arv_delay=-16.0, airtime=173.0, distance=1590.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1256', origin='BZN', dest='MSP', dep=datetime.tim

In [48]:
fp.count()

445827

In [49]:
fp.first()

Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0)

In [50]:
fp.collect()

[Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1248', origin='DTW', dest='LAX', dep=datetime.time(19, 3, 5), dep_delay=0.0, arv=datetime.time(21, 2), arv_delay=-24.0, airtime=249.0, distance=1979.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1251', origin='ATL', dest='GRR', dep=datetime.time(21, 3), dep_delay=5.0, arv=datetime.time(23, 1, 9), arv_delay=-2.0, airtime=92.0, distance=640.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1254', origin='LAX', dest='ATL', dep=datetime.time(22, 5, 6), dep_delay=1.0, arv=datetime.time(5, 4, 7), arv_delay=-13.0, airtime=207.0, distance=1947.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1255', origin='SLC', dest='ATL', dep=datetime.time(17, 0), dep_delay=4.0, arv=datetime.time(22, 1, 3), arv_delay=-16.0, airtime=173.0, distance=1590.0),
 Flight(date=datetime.date(2016, 1, 1), airline='19790', flightnum='1256', origin='BZN', dest='MSP', dep=datetime.tim

#5.Write a Spark Streaming Python Program to Count Error in real Time.

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext("local[2]","StreamingErrorCount")
ssc = StreamingContext(sc,10)

ssc.checkpoint("/streaming")
ds1 = ssc.socketTextStream("localhost", 9999)
count = ds1.flatMap(lambda x:x.split(" ")).filter(lambda word:"Error" in word).map(lambda word:(word,1)).reduceByKey(lambda x,y:x+y)

count.pprint()
ssc.start()
ssc.awaitTermination()